# Post Processing, Visualizing $k_{eff}$ Convergence and Shannon Entropy

In [ ]:
import numpy as np
import openmc
import matplotlib.pyplot as plt

# ----------------------------------------------------------------------------
# 1) Load statepoint and basic parameters
# ----------------------------------------------------------------------------
sp = openmc.StatePoint('statepoint.15.h5')

n_inactive = sp.n_inactive
n_batches  = sp.n_batches
n_active   = n_batches - n_inactive

# ----------------------------------------------------------------------------
# 2) Generation-based data: raw k_generation + Shannon entropy
# ----------------------------------------------------------------------------
k_eff_raw = np.array(sp.k_generation)  # one k-eff per generation
H         = np.array(sp.entropy)       # one Shannon entropy per generation

n_generations = len(k_eff_raw)
gens = np.arange(1, n_generations + 1)

# ----------------------------------------------------------------------------
# 3) Batch-based data: average each batch from k_generation
# ----------------------------------------------------------------------------
# Determine how many generations per batch (assumes even division)
gens_per_batch = n_generations // n_batches

# Average each row => one k value per batch
k_per_batch = np.mean(k_eff_raw.reshape(n_batches, gens_per_batch), axis=1)

# Compute running average over active batches
batch_mean   = np.full(n_batches, np.nan)
batch_stderr = np.full(n_batches, np.nan)

for i in range(n_batches):
    if i < n_inactive:
        # Skip inactive
        continue
    else:
        active_slice = k_per_batch[n_inactive : i+1]
        batch_mean[i] = np.mean(active_slice)
        if len(active_slice) > 1:
            std_sample = np.std(active_slice, ddof=1)
            batch_stderr[i] = std_sample / np.sqrt(len(active_slice))
        else:
            batch_stderr[i] = 0.0

# Plot each batch point at the generation index where the batch ends
batch_x = (np.arange(n_batches) + 1) * gens_per_batch

# ----------------------------------------------------------------------------
# 4) Create the figure and plot
# ----------------------------------------------------------------------------
fig, ax1 = plt.subplots()

# (a) Plot raw generation-based k (no error bars)
line1, = ax1.plot(
    gens, k_eff_raw, 'o-', label='Gen-based k', alpha=0.7
)

# (b) Plot batch-based running average k (with error bars)
line2 = ax1.errorbar(
    batch_x, batch_mean, yerr=batch_stderr,
    fmt='s-', capsize=3, label='Batch-based k', alpha=0.7
)

# Left y-axis for k
ax1.set_xlabel('Generation')
ax1.set_ylabel('k-effective', color='tab:blue')
ax1.tick_params(axis='y', labelcolor='tab:blue')
ax1.grid(True)

# (c) Plot Shannon entropy on a second y-axis
ax2 = ax1.twinx()
line3, = ax2.plot(
    gens, H, 'r-', label='Shannon entropy'
)
ax2.set_ylabel('Shannon Entropy', color='tab:red')
ax2.tick_params(axis='y', labelcolor='tab:red')

# ----------------------------------------------------------------------------
# 5) Combine legends and place them on the middle right
# ----------------------------------------------------------------------------
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()

# Merge them into a single legend
ax2.legend(lines1 + lines2, labels1 + labels2, loc='center right')

plt.title('Gen-based k, Batch-based k, and Shannon Entropy')
plt.tight_layout()
plt.show()


# Post Processing, Visualizing Flux and Fission

In [ ]:
import openmc
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact, IntSlider

# Load the statepoint file
sp = openmc.StatePoint('statepoint.15.h5')

# Function to get data based on tally and score
def get_tally_data(tally_name, score='fission'):
    tally = sp.get_tally(name=tally_name)
    return tally.get_values(scores=[score])

# Retrieve the fission data (change tally name and score if needed)
data_type = 'fission'  # Change this to 'flux', 'fission', or any other score available
plot_title = 'Fission Events'  # Adjust title for visualization
tally_name = 'Fission'  # Adjust the tally name if necessary
data = get_tally_data(tally_name=tally_name, score=data_type)

# Define the mesh size and reshape the data
total_data_points = 300**3
mesh_dimension_size = np.round(np.cbrt(total_data_points)).astype(int)
data_reshaped = data.reshape((mesh_dimension_size, mesh_dimension_size, mesh_dimension_size))

# Determine the data range for consistent color scaling
data_min = data_reshaped.min()
data_max = data_reshaped.max()

# Function to display a single slice
def show_slice(z_slice_index):
    plt.figure(figsize=(8, 6))
    data_2d = data_reshaped[z_slice_index, :, :]

    # Generate the heatmap for the x-y slice with fixed color scale
    plt.imshow(data_2d, origin='lower', cmap='nipy_spectral', vmin=data_min, vmax=data_max)
    plt.colorbar(label='Fission Events')
    plt.title(f'Fission Events in the x-y plane at z index {z_slice_index}')
    plt.xlabel('X-axis')
    plt.ylabel('Y-axis')
    plt.show()

# Create an interactive slider
interact(show_slice, z_slice_index=IntSlider(min=0, max=mesh_dimension_size-1, step=1, value=0));

#Up the contrast of the color bars to make it nicer to look at. 


In [ ]:
import openmc
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact, IntSlider

# Load the statepoint file
sp = openmc.StatePoint('statepoint.15.h5')

# Function to get data based on tally and score
def get_tally_data(tally_name, score='flux'):
    tally = sp.get_tally(name=tally_name)
    return tally.get_values(scores=[score])

# Retrieve the fission data (change tally name and score if needed)
data_type = 'flux'  # Change this to 'flux', 'fission', or any other score available
plot_title = 'Neutron Flux'  # Adjust title for visualization
tally_name = 'Flux'  # Adjust the tally name if necessary
data = get_tally_data(tally_name=tally_name, score=data_type)

# Define the mesh size and reshape the data
total_data_points = 300**3
mesh_dimension_size = np.round(np.cbrt(total_data_points)).astype(int)
data_reshaped = data.reshape((mesh_dimension_size, mesh_dimension_size, mesh_dimension_size))

# Determine the data range for consistent color scaling
data_min = data_reshaped.min()
data_max = data_reshaped.max()

# Function to display a single slice
def show_slice(z_slice_index):
    plt.figure(figsize=(8, 6))
    data_2d = data_reshaped[z_slice_index, :, :]

    # Generate the heatmap for the x-y slice with fixed color scale
    plt.imshow(data_2d, origin='lower', cmap='nipy_spectral', vmin=data_min, vmax=data_max)
    plt.colorbar(label='Neutron Flux')
    plt.title(f'{plot_title} in the x-y plane at z index {z_slice_index}')
    plt.xlabel('X-axis')
    plt.ylabel('Y-axis')
    plt.show()

# Create an interactive slider
interact(show_slice, z_slice_index=IntSlider(min=0, max=mesh_dimension_size-1, step=1, value=0));


# Create a gif File Showing the Fission or Flux

In [ ]:
import openmc
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.animation import FuncAnimation, PillowWriter

# Load the statepoint file
sp = openmc.StatePoint('statepoint.15.h5')

# Function to get data based on tally and score
def get_tally_data(tally_name, score='flux'):
    tally = sp.get_tally(name=tally_name)
    return tally.get_values(scores=[score])

# Retrieve the fission data (change tally name and score if needed)
data_type = 'flux'  # Change this to 'flux', 'fission', or any other score available
plot_title = 'Neutron Flux'  # Adjust title for visualization
tally_name = 'Flux'  # Adjust the tally name if necessary
data = get_tally_data(tally_name=tally_name, score=data_type)

# Define the mesh size and reshape the data
total_data_points = 300**3
mesh_dimension_size = np.round(np.cbrt(total_data_points)).astype(int)
data_reshaped = data.reshape((mesh_dimension_size, mesh_dimension_size, mesh_dimension_size))

# Determine the data range for consistent color scaling
data_min = data_reshaped.min()
data_max = data_reshaped.max()

# Initialize the figure and axis
fig, ax = plt.subplots()

# Function to update the animation for each frame (z-slice)
def update(z_slice_index, plot_title):
    ax.clear()  # Clear previous frame
    data_2d = data_reshaped[z_slice_index, :, :]
    
    # Generate the heatmap for the x-y slice
    im = ax.imshow(data_2d, origin='lower', cmap='viridis', vmin=data_min, vmax=data_max)
    ax.set_title(f'{plot_title} in the x-y plane at z index {z_slice_index}')
    ax.set_xlabel('X-axis')
    ax.set_ylabel('Y-axis')
    return im,

custom_title = "Neutron Flux"

# Create the animation
num_frames = mesh_dimension_size  # One frame per z-slice
ani = FuncAnimation(fig, update, frames=num_frames, fargs=(custom_title,), blit=True, interval=100)

# Save the animation as a video
ani.save('neutron_flux_animation.gif', writer='ffmpeg', fps=10)

# Optionally display the animation as HTML
from IPython.display import HTML
HTML(ani.to_jshtml())

# Close the plot to clean up
plt.close()

# Visualizing Errors

In [ ]:
import openmc
import matplotlib.pyplot as plt
import numpy as np

# Load the statepoint file
sp = openmc.StatePoint('statepoint.15.h5')

# Access the flux tally
flux_tally = sp.get_tally(name='Flux')

# Get the mean and standard deviation for the flux tally
flux_mean = flux_tally.get_values(scores=['flux'], value='mean')
flux_std_dev = flux_tally.get_values(scores=['flux'], value='std_dev')

# Total number of data points
total_data_points = 300**3

# Since the data is from a cubic mesh, we take the cube root to find the size of one dimension
mesh_dimension_size = np.round(np.cbrt(total_data_points)).astype(int)

# Reshape the mean and std_dev data to the mesh dimensions
flux_mean_reshaped = flux_mean.reshape((mesh_dimension_size, mesh_dimension_size, mesh_dimension_size))
flux_std_dev_reshaped = flux_std_dev.reshape((mesh_dimension_size, mesh_dimension_size, mesh_dimension_size))

# Compute the relative error
relative_error = np.zeros_like(flux_mean_reshaped)
nonzero = flux_mean_reshaped > 0
relative_error[nonzero] = flux_std_dev_reshaped[nonzero] / flux_mean_reshaped[nonzero]

# Plot the distribution of relative errors
plt.hist(relative_error[nonzero], bins=500)
plt.title("Distribution of Relative Errors (Flux)")
plt.xlabel("Relative Error")
plt.ylabel("Frequency")
plt.show()

# Access the fission tally
fission_tally = sp.get_tally(name='Fission')

# Get the mean and standard deviation for the fission tally
fission_mean = fission_tally.get_values(scores=['fission'], value='mean')
fission_std_dev = fission_tally.get_values(scores=['fission'], value='std_dev')

# Reshape the mean and std_dev data to the mesh dimensions
fission_mean_reshaped = fission_mean.reshape((mesh_dimension_size, mesh_dimension_size, mesh_dimension_size))
fission_std_dev_reshaped = fission_std_dev.reshape((mesh_dimension_size, mesh_dimension_size, mesh_dimension_size))

# Compute the relative error
relative_error_fission = np.zeros_like(fission_mean_reshaped)
nonzero_fission = fission_mean_reshaped > 0
relative_error_fission[nonzero_fission] = fission_std_dev_reshaped[nonzero_fission] / fission_mean_reshaped[nonzero_fission]

# Plot the distribution of relative errors for fission
plt.hist(relative_error_fission[nonzero_fission], bins=500)
plt.title("Distribution of Relative Errors (Fission)")
plt.xlabel("Relative Error")
plt.ylabel("Frequency")
plt.show()


# Plotting Heat Map of Errors

In [ ]:
import openmc
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider

# Load the statepoint file
sp = openmc.StatePoint('statepoint.15.h5')

# Access the tallies
flux_tally = sp.get_tally(name='Flux')
fission_tally = sp.get_tally(name='Fission')

# Extract mean and standard deviation for flux and fission
flux_data_mean = flux_tally.get_values(scores=['flux'], value='mean')
fission_data_mean = fission_tally.get_values(scores=['fission'], value='mean')
flux_data_std_dev = flux_tally.get_values(scores=['flux'], value='std_dev')
fission_data_std_dev = fission_tally.get_values(scores=['fission'], value='std_dev')

# Define the mesh dimensions (assumed cubic with 300 points in each dimension)
data_points = 300

# Reshape the 1D arrays into 3D volumes
flux_data_mean = flux_data_mean.reshape((data_points, data_points, data_points))
fission_data_mean = fission_data_mean.reshape((data_points, data_points, data_points))
flux_data_std_dev = flux_data_std_dev.reshape((data_points, data_points, data_points))
fission_data_std_dev = fission_data_std_dev.reshape((data_points, data_points, data_points))

# Calculate the relative errors (std_dev / mean)
Relative_errors_flux = flux_data_std_dev / flux_data_mean
Relative_errors_fission = fission_data_std_dev / fission_data_mean

# Convert to percentage
flux_error_percent = Relative_errors_flux * 100
fission_error_percent = Relative_errors_fission * 100

vmin = 0
vmax = 100

# Determine a common color scale range for both images
data_min = min(Relative_errors_flux.min(), Relative_errors_fission.min())
data_max = max(Relative_errors_flux.max(), Relative_errors_fission.max())

def show_slice(z_slice_index):
    # Create a figure with two subplots side by side
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    
    # Rotate the slices to orient them correctly for display
    flux_slice = np.rot90(flux_error_percent[z_slice_index, :, :], 4)
    fission_slice = np.rot90(fission_error_percent[z_slice_index, :, :], 4)
    
    # Plot flux relative error
    im1 = axes[0].imshow(flux_slice, origin='lower', cmap='nipy_spectral', vmin=vmin, vmax=vmax)
    axes[0].set_title(f'Flux Relative Error (Slice {z_slice_index})')
    axes[0].set_xlabel('X-axis')
    axes[0].set_ylabel('Y-axis')
    plt.colorbar(im1, ax=axes[0])
    
    # Plot fission relative error
    im2 = axes[1].imshow(fission_slice, origin='lower', cmap='nipy_spectral', vmin=vmin, vmax=vmax)
    axes[1].set_title(f'Fission Relative Error (Slice {z_slice_index})')
    axes[1].set_xlabel('X-axis')
    axes[1].set_ylabel('Y-axis')
    plt.colorbar(im2, ax=axes[1])
    
    plt.tight_layout()
    plt.show()

# Create an interactive slider to select the z-slice
interact(show_slice, z_slice_index=IntSlider(min=0, max=data_points-1, step=1, value=data_points//2));
